In [81]:
infile = r"./gps.log"

lat_lon = []
keep_phrases = ["Latitude="]

with open(infile) as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    for phrase in keep_phrases:
        if phrase in line:
            lat_lon.append((i, line))
            break

print(lat_lon)

[(469, 'Latitude= 47.275337 Longitude= -2.294765\n'), (505, 'Latitude= 47.275272 Longitude= -2.294804\n'), (538, 'Latitude= 47.275581 Longitude= -2.294675\n'), (570, 'Latitude= 47.275566 Longitude= -2.294688\n'), (607, 'Latitude= 47.275558 Longitude= -2.294687\n'), (644, 'Latitude= 47.275554 Longitude= -2.294681\n'), (681, 'Latitude= 47.275554 Longitude= -2.294676\n'), (717, 'Latitude= 47.275554 Longitude= -2.294672\n'), (753, 'Latitude= 47.275554 Longitude= -2.294673\n'), (789, 'Latitude= 47.275569 Longitude= -2.294674\n'), (844, 'Latitude= 47.275600 Longitude= -2.294645\n'), (875, 'Latitude= 47.275619 Longitude= -2.294637\n'), (911, 'Latitude= 47.275657 Longitude= -2.294625\n'), (947, 'Latitude= 47.275665 Longitude= -2.294627\n'), (983, 'Latitude= 47.275665 Longitude= -2.294657\n'), (1018, 'Latitude= 47.275650 Longitude= -2.294679\n'), (1053, 'Latitude= 47.275619 Longitude= -2.294690\n'), (1089, 'Latitude= 47.275585 Longitude= -2.294715\n'), (1125, 'Latitude= 47.275558 Longitude= -2.

In [104]:
import pandas as pd
df = pd.DataFrame(
    [[i] + coords[len('Latitude= '):-1].split(' Longitude= ') for i, coords in lat_lon[5:]], 
    columns=['line', 'latitude', 'longitude'],
    dtype=float
)
df.line = df.line.astype(int)
df

,line,latitude,longitude
0,644,47.275554,-2.294681
1,681,47.275554,-2.294676
2,717,47.275554,-2.294672
3,753,47.275554,-2.294673
4,789,47.275569,-2.294674
...,...,...,...
63,3036,47.275516,-2.294673
64,3072,47.275550,-2.294675
65,3107,47.275573,-2.294672
66,3144,47.275577,-2.294669


In [105]:
df.line

0      644
1      681
2      717
3      753
4      789
      ... 
63    3036
64    3072
65    3107
66    3144
67    3181
Name: line, Length: 68, dtype: int64

In [106]:
df.line

0      644
1      681
2      717
3      753
4      789
      ... 
63    3036
64    3072
65    3107
66    3144
67    3181
Name: line, Length: 68, dtype: int64

In [116]:
def enrich(df, offset, extractor):
    return df.line.apply(lambda nb: extractor(lines[nb + offset]))

df['time'] = enrich(df, 9, lambda line: line.split(" = ")[1].rstrip()).astype('int')
df['nb_sat'] = enrich(df, 26, lambda line: line.split(" = ")[1].rstrip()).astype('int')
df['hdop'] = enrich(df, 27, lambda line: line.split(" = ")[1].rstrip()).astype('int')
df

,line,latitude,longitude,time,nb_sat,hdop
0,644,47.275554,-2.294681,14051300,6,224
1,681,47.275554,-2.294676,14051600,6,224
2,717,47.275554,-2.294672,14051900,6,224
3,753,47.275554,-2.294673,14052200,6,224
4,789,47.275569,-2.294674,14052500,6,224
...,...,...,...,...,...,...
63,3036,47.275516,-2.294673,14083700,0,9999
64,3072,47.275550,-2.294675,14084000,0,9999
65,3107,47.275573,-2.294672,14084300,0,9999
66,3144,47.275577,-2.294669,14084600,0,9999


## Plotting on map
https://pypi.org/project/gmplot/

In [117]:
BBox = (df.longitude.min(),   df.longitude.max(),      
        df.latitude.min(), df.latitude.max())
BBox

(-2.294968, -2.294507, 47.275455, 47.275665)

In [118]:
import gmplot
from IPython.display import IFrame

# Create the map plotter:
apikey = open('api.key', 'r').readline().rstrip() # (your API key here)
def getMap(df):
    gmap = gmplot.GoogleMapPlotter(df.latitude.mean(), df.longitude.mean(), 25, apikey=apikey, map_type='hybrid',
                                   fit_bounds={
                                       'north': df.latitude.max(), 
                                       'south': df.latitude.min(), 
                                       'east': df.longitude.max(), 
                                       'west': df.longitude.min()
                                   })

    gmap.plot(df.latitude, df.longitude, color='cornflowerblue', edge_width=10)

    gmap.scatter(df.latitude, df.longitude, color='#3B0B39', size=0.4, marker=False)

    # Mark a hidden gem:
    # gmap.marker(37.770776, -122.461689, color='cornflowerblue')

    # Draw the map:
    gmap.draw('map.html')
    return IFrame(src='./map.html', width=600, height=500)

getMap(df)

In [120]:
from ipywidgets import *
def update(i = IntSlider(0, 0, len(df)), j = IntSlider(len(df), 0, len(df))):
    # print(df.iloc[i:j].hdop)
    return getMap(df.iloc[i:j])

interact(update);

interactive(children=(IntSlider(value=0, description='i', max=68), IntSlider(value=68, description='j', max=68…